In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy
from collections import defaultdict, Counter
from itertools import chain, combinations

In [ ]:
torch.backends.mps.is_available()

# Random Variables
## Probability Space

* The sample space $\Omega$: the set of all possible outcomes of the experiment
  * Ex.) $\Omega$ of two successive coin tosses: $\{ HH, HT, TH, TT \}$
* The event space $\mathcal{F}$: the set of events (the power set of $\Omega$: $2^{\Omega}$)
  * event: the subset of the sample space $\Omega$
* The probability or probability measure $P$: each event $A \in \mathcal{F}$, $P(A)$ measures the probability or degree of belief that the event will occur
  * $P(A)$ is called the probability of $A$
* These three properties are called the **Axioms of Probability**

In [ ]:
np.random.seed(219)
np.set_printoptions(precision=4)

In [ ]:
def powerset(iterable):
  "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
  s = list(iterable)
  ps = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
  ps = [set(elem) for elem in ps]
  return ps

In [ ]:
def probability_measure(event_space, event):
  assert event in event_space
  size_of_sample_space = len(max(event_space))
  return len(event) / size_of_sample_space

In [ ]:
# sample_space = {f'{i}{j}' for i in ['H', 'T'] for j in ['H', 'T']}
# sample_space = {f'{i}{j}{k}' for i in ['H', 'T'] for j in ['H', 'T'] for k in ['H', 'T']}
sample_space = {f'{i}{j}' for i in ['H', 'T'] for j in range(1, 7)}
print(sample_space)

In [ ]:
event_space = powerset(sample_space)
print(len(event_space))

In [ ]:
probability_measure(event_space, {'HH', 'HT', 'TH'})

In [ ]:
probability_measure(event_space, {'HHH', 'HTH', 'THT'})

Probability measure

$P(\cup_{i=1}^{\infty} A_{i}) = \Sigma{}_{i=1}^{\infty} P(A_{i})$, if $A_{1}, A_{2}, \ldots$ are events that are *mutually exclusive*

In [ ]:
probability_measure(event_space, {'HH', 'TH'}) + probability_measure(event_space, {'HT'})

## Random Variables

* **Definition**: Given a probability space $(\Omega, \mathcal{F}, P)$, let $\Omega$ be a sample space and let $X: \Omega \rightarrow \mathbb{R}$ be a function from the sample space to the real line. Then $X$ is called a *random variable*
  * Notation: upper case letters $X(\omega)$ or more simply $X$
    * Random variables depend on random outcome $\omega$
* Ex.) two successive coin tosses, $X$: counting the number of heads
  * $\Omega = \{ HH, HT, TH, TT \}$
  * $X(HH) = 2, \ X(HT) = 1, \ X(TH) = 1, \ X(TT) = 0$

In [ ]:
sample_space = {f'{i}{j}' for i in ['H', 'T'] for j in ['H', 'T']}

In [ ]:
def random_variable_X(sample_space, event):
  # X: the number of heads
  assert event in sample_space
  return event.count('H')

In [ ]:
event = 'HH'
print(f'random variable X of event {event}: {random_variable_X(sample_space, event)}')

### Probability of random variables

* Consider triple successive coin tosses
  * $\Omega = \{ HHH, \, HHT, \, HTH, \, HTT, \, THH, \, THT, \, TTH, \, TTT \}$
  * Let $X$: the number of heads
  * Let $Y$: head at first trial: 1 ; tail at first trial: 0
  
|   | HHH | HHT | HTH | HTT | THH | THT | TTH | TTT |
|---|---|---|---|---|---|---|---|---|
| $X$ | 3 | 2 | 2 | 1 | 2 | 1 | 1 | 0 |
| $Y$ | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 |
| $P(\omega)$ | 1/8 | 1/8 | 1/8 | 1/8 | 1/8 | 1/8 | 1/8 | 1/8 |

In [ ]:
def random_variable_X(sample_space, event):
  # X: the number of heads
  assert event in sample_space
  return event.count('H')

In [ ]:
def random_variable_Y(sample_space, event):
  # Y: head at first trial: 1; tail at first trial: 0
  assert event in sample_space
  return 1 if event[0] == 'H' else 0

In [ ]:
sample_space = {f'{i}{j}{k}' for i in ['H', 'T'] for j in ['H', 'T'] for k in ['H', 'T']}

In [ ]:
sample_space

In [ ]:
event = 'TTH'
print(f'random variable X of event {event}: {random_variable_X(sample_space, event)}')
print(f'random variable Y of event {event}: {random_variable_Y(sample_space, event)}')

In [ ]:
# probability distribution of X
PX = defaultdict(list)
for event in sorted(list(sample_space)):
  X = random_variable_X(sample_space, event)
  print(f'{event}: {X}')
  PX[X].append(event)

In [ ]:
# probability distribution of Y
for event in sorted(list(sample_space)):
  Y = random_variable_Y(sample_space, event)
  print(f'{event}: {Y}')

## Probability Distributions

* **Definition**: Given a random variable $X: \Omega \rightarrow \mathbb{R}$, let $A \subset \mathbb{R}$ be a any subset of the real line. Then probability distribution $P_{X} (A)$ is defined by
$$P_{X}(A) = P(\{ \omega \in \Omega: X(\omega) \in A \})$$

* Discrete probability distribution (probability mass function) of $X$
$$P(X=k) = P(\{ \omega \in \Omega: X(\omega) = k \})$$
* Continuous probability distribution (probability density function) of $X$
$$P(a \leq X \leq b) = P(\{ \omega \in \Omega: a \leq X(\omega) \leq b \})$$

In [ ]:
# probability distribution of X
PX = defaultdict(list)
for event in sorted(list(sample_space)):
  X = random_variable_X(sample_space, event)
  PX[X].append(event)
PX

In [ ]:
# probability distribution of Y
PY = defaultdict(list)
for event in sorted(list(sample_space)):
  X = random_variable_Y(sample_space, event)
  PY[X].append(event)
PY

In [ ]:
def draw_distribution(PX):
  x = []  # random variable
  y = []  # count
  for X, events in PX.items():
    x.append(X)
    y.append(len(events))
  x = np.array(x)
  y = np.array(y, dtype=np.float32)
  total_events = y.sum()
  y /= total_events

  plt.bar(x, y)
  if len(x) < 10:
    xticks = [str(xx) for xx in x]
    plt.xticks(x, xticks)
  else:
    tick_size = int((np.max(x) - np.min(x)) / 10)
    xrange = list(range(np.min(x), np.max(x), tick_size))
    xticks = list(map(str, xrange))
    plt.xticks(xrange, xticks)
  plt.show()

  return np.concatenate((x[:, None], y[:, None]), axis=-1)

In [ ]:
px = draw_distribution(PX)

In [ ]:
py = draw_distribution(PY)

### Example) throwing a dic twice in succession

* ranom variable $Z$: the summation of two dice face

In [ ]:
sample_space = {f'{i}-{j}' for i in range(1, 7) for j in range(1, 7)}
print(sample_space)

In [ ]:
def random_variable_Z(sample_space, event):
  # Z: summation of two dice
  assert event in sample_space
  i, j = map(int, event.split('-'))
  return i + j

In [ ]:
event = '6-2'
print(f'random variable Z of event {event}: {random_variable_Z(sample_space, event)}')

In [ ]:
# probability distribution of Z
PZ = defaultdict(list)
for event in sorted(list(sample_space)):
  X = random_variable_Z(sample_space, event)
  PZ[X].append(event)
PZ

In [ ]:
pz = draw_distribution(PZ)

### Example) throwing a die three times in a row

* ranom variable $W$: the product of three dice face

In [ ]:
sample_space = {f'{i}-{j}-{k}' for i in range(1, 7) for j in range(1, 7) for k in range(1, 7)}
print(len(sample_space))

In [ ]:
def random_variable_W(sample_space, event):
  # Z: summation of two dice
  assert event in sample_space
  i, j, k = map(int, event.split('-'))
  return i * j * k

In [ ]:
event = '4-3-2'
print(f'random variable W of event {event}: {random_variable_W(sample_space, event)}')

In [ ]:
# probability distribution of W
PW = defaultdict(list)
for event in sorted(list(sample_space)):
  X = random_variable_W(sample_space, event)
  PW[X].append(event)
# PW

In [ ]:
pw = draw_distribution(PW)

### Properties of discrete probability distributions

* Normalization:
$$\sum_{x \in \mathcal{X}} P(x) = 1$$

In [ ]:
print(f'the summation of distribution of P(X): {px[:, 1].sum():.4f}')
print(f'the summation of distribution of P(Y): {py[:, 1].sum():.4f}')
print(f'the summation of distribution of P(Z): {pz[:, 1].sum():.4f}')
print(f'the summation of distribution of P(W): {pw[:, 1].sum():.4f}')

In [ ]:
a = 1

In [ ]:
a = 1

In [ ]:
a = 1

In [ ]:
a = 1

In [ ]:
a = 1

In [ ]:
a = 1

In [ ]:
a = 1

## Exepctation of a Random Variable

* Discrete random variable $X$ with PMF $P_{X}(x)$
  * Expectation or expected value of a random variable $X$
$$\mathbb{E}[X] = \sum_{x \in \mathcal{X}} x P(x)$$

* Continuous random variable $X$ with PDF $p_{X}(x)$
  * Expectation or expected value of a random variable $X$
$$\mathbb{E}[X] = \int_{\mathcal{X}} x p(x) \mathrm{d}x$$

In [ ]:
# expectation of random variable
def expectation(px, message=False):
  s = 0.
  for x, p in px:
    if message:
      print(f'random variable: {x} / probability: {p:.3f}')
    s += x * p
  print(f'expectation of a random variable: {s:.3f}')
  return s

In [ ]:
_ = expectation(px)

In [ ]:
_ = expectation(py)

In [ ]:
_ = expectation(pz)

In [ ]:
_ = expectation(pw)

### Expectation of functions

* Discrete random variable $X$
  * PMF $P_{X}(s)$ and an arbitrary function $g: \mathbb{R} \rightarrow \mathbb{R}$
  * Expectation or expected value of $g(s)$
$$\mathbb{E} [g(X)] = \sum_{x \in \mathcal{X}} P_{X}(x) g(x)$$

* Continuous random variable $X$ with PDF $p_{X}(s)$
  * Expectation or expected value of $g(x)$
$$\mathbb{E} [g(X)] = \int_{-\infty}^{\infty} p_{X}(x) g(x)$$

In [ ]:
# expectation of a fucntion
def expectation_of_functions(px, gx, message=False):
  s = 0.
  for x, p in px:
    if message:
      print(f'random variable: {x} / probability: {p:.3f}')
    s += gx(float(x)) * p
  return s

In [ ]:
# gx = lambda x: 2 * x
gx = lambda x: x * x
gx = lambda x: np.sin(x / np.pi)
gx = lambda x: np.exp(-x / 10)

In [ ]:
# mu = pw.prod(axis=1).sum()

In [ ]:
mu = expectation_of_functions(px, gx)  # E(X) = 1.5
print(f'expectation of a random variable: {mu:.3f}')

In [ ]:
mu = expectation_of_functions(py, gx)  # E(Y) = 0.5
print(f'expectation of a random variable: {mu:.3f}')

In [ ]:
mu = expectation_of_functions(pz, gx)  # E(Z) = 7.0
print(f'expectation of a random variable: {mu:.3f}')

In [ ]:
mu = expectation_of_functions(pw, gx)  # E(W) = 42.875
print(f'expectation of a random variable: {mu:.3f}')

### Properties of expectation

* $\mathbb{E}[a] = a$ for any constant $a \in \mathbb{R}$
* $\mathbb{E}[ a g(X) ]= a \mathbb{E}[ g(X) ]$ for any constant $a \in \mathbb{R}$
* *Linearity of expectation*: $\mathbb{E} [a f(X) + b g(X)] = a \mathbb{E}[f(X)] + b \mathbb{E}[g(X)]$

In [ ]:
# property1
gx = lambda x: 3
mu = expectation_of_functions(px, gx)  # E(X) = 1.5
print(f'expectation of a random variable: {mu:.3f}')

In [ ]:
# property2
a = 3
gx = lambda x: np.sin(x / np.pi)
fx = lambda x: a * gx(x)
mu = expectation_of_functions(px, fx)  # E(X) = 1.5
mu_r = expectation_of_functions(px, gx)
print(f'left: {mu:.4f} / right: {a * mu_r:.4f}')

In [ ]:
# property3: linearity of expectation
a, b = 3, 2
fx = lambda x: np.sin(x / np.pi)
gx = lambda x: np.exp(-x / 10)
lin_fg = lambda x: a * fx(x) + b * gx(x)
mu_l = expectation_of_functions(px, lin_fg)
mu_r1 = expectation_of_functions(px, fx)
mu_r2 = expectation_of_functions(px, gx)
print(f'left: {mu_l:.4f} / right: {a * mu_r1 + b * mu_r2:.4f}')

### Variance of a random variable

* **Definition**
$$\mathrm{Var}[X] = \mathbb{E}[ (X - \mathbb{E}[X])^{2} ]$$
* *Standard deviation*: the square root of the variance
$$\sigma = \sqrt{\mathrm{Var}[X]}$$
* Alternate expression
$$\begin{array}{rl}
\mathbb{E} [ (X - \mathbb{E}[X])^{2} ]
&= \mathbb{E} [ X^{2} - 2X\mathbb{E}[X] + \mathbb{E}[X]^{2} ]\\
&= \mathbb{E} [ X^{2} ] - 2\mathbb{E}[X] \mathbb{E}[X] + \mathbb{E}[X]^{2}\\
&= \mathbb{E}[X^{2}] - \mathbb{E}[X]^{2}
\end{array}$$

In [ ]:
# variance of a random variable
def variance(px, message=False):
  s = 0.
  ss = 0.
  for x, p in px:
    s += x * p
    ss += x**2 * p
  return ss - s**2

In [ ]:
# mu = pw.prod(axis=1).sum()
# ((pw[:, 0] - mu)**2 * pw[:, 1]).sum()

In [ ]:
var = variance(px)
print(f'variance of a random variable: {var:.4f}')

In [ ]:
var = variance(py)
print(f'variance of a random variable: {var:.4f}')

In [ ]:
var = variance(pz)
print(f'variance of a random variable: {var:.4f}')

In [ ]:
var = variance(pw)
print(f'variance of a random variable: {var:.4f}')

### Properties of variance

* $\mathrm{Var}(X) \geq 0 $
* $\mathrm{Var}(a) = 0$, $a$ is a constant
* $\mathrm{Var}(X + a) = \mathrm{Var}(X)$
* $\mathrm{Var}(aX) = a^{2} \mathrm{Var}(X)$
* $\mathrm{Var}(aX \pm bY) = a^{2} \mathrm{Var}(X) + b^{2} \mathrm{Var}(X) \pm 2ab \mathrm{Cov}(X, Y)$
* $\mathrm{Var}(f(x)) = \mathbb{E} [ ( f(x) - \mathbb{E}[ f(x) ] )^{2} ]$

In [ ]:
# property3
a = 3
_px = deepcopy(px)
_px[:, 0] = a + _px[:, 0]
print(f'variance of X: {variance(px)}')
print(f'variance of translated X: {variance(_px)}')

In [ ]:
# property4
a = 3
_px = deepcopy(px)
_px[:, 0] = a * _px[:, 0]
print(f'variance of X: {variance(px)}')
print(f'variance of scaled X: {variance(_px)}')

In [ ]:
0.75 * 9 

## Exmaple of discrete probability distributions

### Uniform distribution: (random variable $X$ with $k$ different states)
$$P(X = x_{i}) = \frac{1}{k}, \quad
\sum_{i} P(X = x_{i}) = \sum_{i} \frac{1}{k} = \frac{k}{k} = 1$$

In [ ]:
# using `numpy.random.randint`
N = 100000
x = np.random.randint(low=0, high=10, size=N)
print(x[:100])

In [ ]:
plt.hist(x, density=True)

In [ ]:
print(f'mean of discrete uniform distribution [0, 9]: {x.mean():.4f}')
print(f'variance of discrete uniform distribution [0, 9]: {x.var():.4f}')

In [ ]:
a = 1

### Bernoulli distributions

* probability mass function
$$f(k; p) = \left\{ \begin{array}{ll}
p & \textrm{if} \, k = 1,\\
1 - p & \textrm{if} \, k=0
\end{array} \right.$$
* mean: $p$
* variance: $p(1-p)$

In [ ]:
p = 0.7
x = np.random.binomial(1, p, size=10000)

In [ ]:
plt.hist(x, density=True)

In [ ]:
print(f'mean of Bernoulli distribution: {x.mean():.4f}')
print(f'variance of Bernoulli distribution: {x.var():.4f}')

### Binomial distributions

<img width="300" src="https://user-images.githubusercontent.com/11681225/184646169-32cb1469-96a5-46bd-98e7-26d65c8b76e3.png">

* probability mass function
$$\begin{array}{l}
&f(k; n, p) = \mathrm{Pr}(k; n, p) \\
& \quad = \mathrm{Pr}(X = k) = \left( \begin{array}{c}
n\\
k
\end{array} \right)
p^{k} (1 - p)^{n-k}
\end{array}$$
for $k = 0, 1, \ldots, n$, where $$\left( \begin{array}{c}
n\\
k
\end{array} \right) = \frac{n!}{k! (n-k)!}$$
* mean: $np$
* variance: $np(1-p)$

In [ ]:
n = 40
p = 0.5
x = np.random.binomial(n, p, size=10000)

In [ ]:
left = min(x)
right = max(x)
plt.hist(x, density=True, bins=range(left, right + 2, 1))
plt.show()

In [ ]:
print(f'mean of binomial distribution ({n}, {p}): {x.mean():.4f}')  # mean: n * p
print(f'variance of binomial distribution ({n}, {p}): {x.var():.4f}')  # var: n * p * (1 - p)

### Multinoulli distributions

* Single discrete variable with $k$ different states, where $k$ is finite
* Sample space: $\Omega = \{1, \ldots, ,k \}$
* Probability distribution, $X \sim \mathrm{Cat}(p_{1}, \ldots, p_{k})$
  * $P(X = i) = p_{i}$, such that $\sum p_{i} = 1$
* Probability mass function
$$f(x = i | \boldsymbol{p}) = p_{i}$$
where $\boldsymbol{p} = (p_{1}, \ldots, p_{k})$

In [ ]:
n = 10000
num_category = 10
p = np.random.choice(100, size=num_category) / 50
p = torch.softmax(torch.tensor(p), dim=0).numpy()
p = np.sort(p)
x = np.random.multinomial(n=1, pvals=p, size=10)

In [ ]:
x = np.random.multinomial(1, p, size=n)
x = np.argmax(x, axis=1)

In [ ]:
plt.hist(x, density=True, bins=range(0, num_category, 1))
plt.show()

## Exmaple of continuous probability distributions

### Uniform distribution

<img width="200" src="https://user-images.githubusercontent.com/11681225/184651948-0fe9f5e8-fb05-40cb-81be-b606faa4d1c0.png">

* Probability density function
$$f(x) = \left\{ \begin{array}{ll}
\frac{1}{b-a} & \textrm{for} \ a \leq x \leq b,\\
0 & \textrm{for} \ x < a \ \textrm{or} \ x > b
\end{array} \right.$$
* mean: $(a + b) / 2$
* variance: $\frac{(b - a)^{2}}{12}$

In [ ]:
a, b = 3, 7
x = np.random.uniform(low=a, high=b, size=10000)

In [ ]:
plt.hist(x, density=True, bins=20)
plt.show()

In [ ]:
print(f'theoretical mean of x: {(a + b) / 2:.4f}')
print(f'emperical mean of x: {x.mean():.4f}')

In [ ]:
print(f'theoretical variance of x: {(b - a)**2 / 12:.4f}')
print(f'emperical mean of x: {x.var():.4f}')

### Normal (Gaussian) distribution

<img width="300" src="https://user-images.githubusercontent.com/11681225/184653267-931ea449-36de-45c3-981c-d1fdeef14c38.png">

* probability density function
$$\mathcal{N}(x; \mu, \sigma^{2})
= \sqrt{\frac{1}{2\pi \sigma^{2}}}
\exp \left( - \frac{(x - \mu)^{2}}{2 \sigma^{2}} \right)$$
* mean: $\mu$
* variance: $\sigma^{2}$

In [ ]:
mu = -2
sigma = np.sqrt(0.5)
x = np.random.normal(loc=mu, scale=sigma, size=100000)

In [ ]:
plt.hist(x, density=True, bins=100)
plt.show()

In [ ]:
plt.hist(x, density=True, cumulative=True, histtype='step', bins=100)
plt.show()

In [ ]:
print(f'theoretical mean of x: {mu:.4f}')
print(f'emperical mean of x: {x.mean():.4f}')

In [ ]:
print(f'theoretical variance of x: {sigma**2:.4f}')
print(f'emperical variance of x: {x.var():.4f}')

# Two Random Variables

## Joint Probability Mass Functions

|   | $y_{1}$ | $y_{2}$ | $y_{3}$ | $P(X)$ |
|:---:| --- | --- | --- | --- |
| $x_{1}$ | 4/50 | 6/50 | 10/50 | 20/50 |
| $x_{2}$ | 6/50 | 9/50 | 15/50 | 30/50 |
| $P(Y)$ | 10/50 | 15/50 | 25/50 | 50/50 |

* **Joint probability distribution**: the probability distribution of multiple random variables
$$P_{XY}(x, y) = P(X = x, Y= y)$$
* **Marginal probability distribution**: the marginal probability is the probability of a single event occurring, independent of other events
$$\forall x \in X, \, P_{X}(x) = \sum_{y \in \mathcal{Y}} P_{XY}(x, y) \quad 
\forall y \in Y, \, P_{Y}(y) = \sum_{x \in \mathcal{X}} P_{XY}(x, y)$$

In [ ]:
n = 1000000
p = np.array([0.4, 0.6])
x = np.random.multinomial(1, p, size=n)
x = np.argmax(x, axis=1)

q = np.array([0.2, 0.3, 0.5])
y = np.random.multinomial(1, q, size=n)
y = np.argmax(y, axis=1)

xy = np.concatenate((x[:, None], y[:, None]), axis=1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
#x, y = np.random.rand(2, 100) * 4
hist, xedges, yedges = np.histogram2d(x, y, bins=3, range=[[0, 1], [0, 2]])

# Construct arrays for the anchor positions of the 16 bars.
xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25, indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = 0

# Construct arrays with the dimensions for the 16 bars.
dx = dy = 0.5 * np.ones_like(zpos)
dz = hist.ravel()

ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')

plt.show()

In [ ]:
xy

In [ ]:
XY = xy.tolist()
XY = [tuple(item) for item in XY]

In [ ]:
counting = Counter(XY)

In [ ]:
counting

In [ ]:
prob_dist = np.zeros([2, 3])
for i in range(2):
  for j in range(3):
    prob_dist[i, j] = counting[(i, j)] / n

In [ ]:
prob_dist

In [ ]:
prob_dist
px = prob_dist.sum(axis=1)
py = prob_dist.sum(axis=0)

In [ ]:
px

In [ ]:
py

## Independence

* Two random variables $X$ and $Y$ are independent
$$\forall x \in X, y \in Y, \, P_{XY}(x, y) = P_{X}(x)P_{Y}(y)$$
* from conditional probability formula
$$\begin{array}{rl}
P_{Y|X}(y|x) &= \frac{P_{XY}(x, y)}{P_{X}(x)} \\
&= \frac{P_{X}(x) P_{Y}(y)}{P_{X}(x)}
= P_{Y}(y), \quad \textrm{whenever } P_{X}(x) \neq 0
\end{array}$$

* Notation: $X \perp Y$

In [ ]:
for i in range(2):
  for j in range(3):
    print(f'{prob_dist[i, j]:.3f} / {px[i] * py[j]:.3f}')
    print(f'{np.allclose(prob_dist[i, j], px[i] * py[j], atol=1e-3)}')

## Joint Probability Density Functions

<img width="300" src="https://user-images.githubusercontent.com/11681225/186937529-bf60ee92-5152-4b67-8610-0545ee2b3b5c.png">

In [ ]:
mean = np.array([2.0, -1.0])
cov = np.array([ [0.5, 0.7],
                 [0.7, 3.0] ])
xy = np.random.multivariate_normal(mean, cov, size=100000)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.scatter(xy[:, 0], xy[:, 1])
plt.show()

In [ ]:
X = xy[:, 0]
print(f'mean of X: {X.mean():.4f} / variance of X: {X.var():.4f}')
plt.hist(X, bins=100, density=True)
plt.show()

In [ ]:
Y = xy[:, 1]
print(f'mean of Y: {Y.mean():.4f} / variance of Y: {Y.var():.4f}')
plt.hist(Y, bins=100, density=True)
plt.show()

In [ ]:
np.cov(X, Y)

In [ ]:
xy.mean()

### Covariance

$$\mathrm{Cov}[X, Y] = \mathbb{E}[ (X - \mathbb{E}[X]) (Y - \mathbb{E}[Y]) ]$$
$$\mathrm{Cov}[X, Y] = \mathbb{E}[XY] - \mathbb{E}[X] \mathbb{E}[Y]$$

In [ ]:
cov = (X * Y).mean() - (X.mean() * Y.mean())
print(f'covariance of X and Y: {cov:.4f}')

#### Properties of covariance

* $\mathbb{E}[ f(X, Y) + g(X, Y) ] = \mathbb{E}[f(X, Y)] + \mathbb{E}[g(X, Y)]$
* $\mathrm{Var}[aX \pm bY] = a^{2} \mathrm{Var}[X] + b^{2} \mathrm{Var}[Y] \pm 2ab \mathrm{Cov}[X, Y]$
* If $X$ and $Y$ are independent, then $\mathrm{Cov}[X, Y] = 0$
* If $X$ and $Y$ are independent, then $\mathbb{E}[f(X) g(Y)] = \mathbb{E}[f(X)] \mathbb{E}[g(Y)]$
* $\mathrm{Cov}[X, X] = \mathrm{Var}[X]$

In [ ]:
# property 1
f = 2 * X + Y
g = X * Y
left = (f + g).mean()
right = f.mean() + g.mean()
print(f'left: {left:.4f} / right: {right:.4f}')

In [ ]:
# property 2
a, b = 2, 3
c = a * X + b * Y
left = c.var()
right = a**2 * X.var() + b**2 * Y.var() + 2 * a * b * np.cov(X, Y)[0, 1]
print(f'left: {left:.5f} / right: {right:.5f}')
print(np.allclose(left, right))

### The sign of the covariance

<img width="600" alt="Covariance_trends1" src="https://user-images.githubusercontent.com/11681225/186956292-d3af8aa1-01d1-4870-ac55-97d106bd32b5.png">

In [ ]:
mean = np.array([-3.0, 2.0])
cov = np.array([ [0.5, 0.6],
                 [0.6, 1.0] ])
xy1 = np.random.multivariate_normal(mean, cov, size=100000)

In [ ]:
mean = np.array([4.0, 2.0])
cov = np.array([ [0.5, -0.6],
                 [-0.6, 1.0] ])
xy2 = np.random.multivariate_normal(mean, cov, size=100000)

In [ ]:
xy = np.concatenate((xy1, xy2), axis=0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.scatter(xy[:, 0], xy[:, 1])
plt.show()

In [ ]:
np.set_printoptions(precision=4)
np.cov(xy[:, 0], xy[:, 1])

In [ ]:
X = xy[:, 0]
print(f'mean of X: {X.mean():.4f} / variance of X: {X.var():.4f}')
plt.hist(X, bins=100, density=True)
plt.show()

In [ ]:
Y = xy[:, 1]
print(f'mean of Y: {Y.mean():.4f} / variance of Y: {Y.var():.4f}')
plt.hist(Y, bins=100, density=True)
plt.show()

In [ ]:
print(f'{(X * Y).mean():.4f}')

In [ ]:
print(f'{X.mean() * Y.mean():.4f}')

# Multiple Random Variables

## Random Vectors

* We have $n$ random variables, $X_{1}, X_{2}, \ldots, X_{n}$
* Considered these random variables as a vector, $X = [X_{1}, X_{2}, \ldots, X_{n} ]^{\top}$
* Random vector
$$X = \left[ \begin{array}{c}
X_{1} \\
X_{2} \\
\vdots \\
X_{n}
\end{array} \right]$$

In [ ]:
N = 100000
mean = np.array([2.0, -1.0, 1.0])
cov = np.array([ [0.5, 0.7, -0.3],
                 [0.7, 3.0, 0.8],
                 [-0.3, 0.8, 2.0] ])
X = np.random.multivariate_normal(mean, cov, size=N)

## Covariance Matrix of a Random Vector

* Given a random vector $X: \Omega \rightarrow \mathbb{R}^{n}$
* Covariance matrix:
$$ \begin{array}{rl}
\boldsymbol{\Sigma} &= 
\left[ \begin{array}{ccc}
\mathrm{Cov}[X_{1}, X_{1}] & \cdots & \mathrm{Cov}[X_{1}, X_{n}] \\
\vdots & \ddots & \vdots \\
\mathrm{Cov}[X_{n}, X_{1}] & \cdots & \mathrm{Cov}[X_{n}, X_{n}]
\end{array} \right] \\
&=
\mathbb{E}[ XX^{\top} ] - \mathbb{E}[X] \mathbb{E}[X]^{\top}
= \cdots
= \mathbb{E}[ (X - \mathbb{E}[X]) (X - \mathbb{E}[X])^{\top} ]
\end{array}$$

In [ ]:
X.mean(axis=0, keepdims=True)

In [ ]:
X1 = X[:, 0]
X2 = X[:, 1]
X3 = X[:, 2]
cov1 = np.cov([X1, X2, X3])

In [ ]:
cov2 = (np.matmul(X.T, X) / N) - np.matmul(X.mean(axis=0, keepdims=True).T, X.mean(axis=0, keepdims=True))

In [ ]:
cov3 = np.matmul((X - X.mean(axis=0, keepdims=True)).T, (X - X.mean(axis=0, keepdims=True))) / N

In [ ]:
print(f'cov1 and cov2 are the same: {np.allclose(cov1, cov2)}')
print(f'cov1 and cov3 are the same: {np.allclose(cov1, cov3)}')
print(f'cov2 and cov3 are the same: {np.allclose(cov2, cov3)}')

## Multivariate Normal Distribution

<img width="300" src="https://user-images.githubusercontent.com/11681225/186937529-bf60ee92-5152-4b67-8610-0545ee2b3b5c.png">

* Probability density function of the multivariate normal (Gaussian) distribution
$$\mathcal{N}(\boldsymbol{x}; \boldsymbol{\mu}, \boldsymbol{\Sigma})
= \sqrt{ \frac{1}{(2 \pi)^{n} \mathrm{det}( \boldsymbol{\Sigma} )} }
\exp \left(
- \frac{1}{2} (\boldsymbol{x} - \boldsymbol{\mu})^{\top}
\boldsymbol{\Sigma}^{-1} (\boldsymbol{x} - \boldsymbol{\mu}) \right)$$

* Two parameters:
  * mean (or expectation): $\boldsymbol{\mu} \in \mathbb{R}^{n}$
  * covariance matrix: $\boldsymbol{\Sigma} \in \mathbb{R}^{n \times n}$

In [ ]:
N = 100000
mean = np.array([2.0, -1.0, 1.0])
cov = np.array([ [0.5, 0.7, -0.3],
                 [0.7, 3.0, 0.8],
                 [-0.3, 0.8, 2.0] ])
X = np.random.multivariate_normal(mean, cov, size=N)

# Bayes' Theorem

## Posterior, Likelihood and Prior

* Let $\boldsymbol{\theta}$ be the model parameters and $\mathcal{D}$ be the data
<img width="600" alt="posterior, likelihood and prior" src="https://user-images.githubusercontent.com/11681225/187081981-d9d328ef-e440-4f9e-87b1-70a54acf2bd2.png">
* **posterior**: the model probability obtained after observing the data $\mathcal{D}$
* **prior**: the model probability available before observing any data $\mathcal{D}$
* **likelihood** [$\mathcal{L}(\boldsymbol{\theta}|\boldsymbol{x}) = P(X=\boldsymbol{x} | \boldsymbol{\theta})$]: how probable the observed data set is for different parameters
  * the function of the model $\boldsymbol{\theta}$ given data

## Maximum Likelihood Estimation

$$\hat{\boldsymbol{\theta}}
= \mathop{\arg \max}\limits_{\boldsymbol{\theta}}
p(\boldsymbol{\mathcal{D}} | \boldsymbol{\theta}) 
= \mathop{\arg \max}\limits_{\boldsymbol{\theta}}
\prod_{i=1}^{n} p(\boldsymbol{x}^{(i)} | \boldsymbol{\theta} )$$

## Maximum Log-Likelihood

$$\hat{\boldsymbol{\theta}}
= \mathop{\arg \max}\limits_{\boldsymbol{\theta}}
\log p(\boldsymbol{\mathcal{D}} | \boldsymbol{\theta}) 
= \mathop{\arg \max}\limits_{\boldsymbol{\theta}}
\log \left[ \prod_{i=1}^{n} p(\boldsymbol{x}^{(i)} | \boldsymbol{\theta} ) \right]
= \mathop{\arg \max}\limits_{\boldsymbol{\theta}}
\sum_{i=1}^{n} \log p(\boldsymbol{x}^{(i)} | \boldsymbol{\theta} )
$$

In [ ]:
N = 10000
mean, std = -3.0, 2.0
x = np.random.normal(loc=mean, scale=std, size=(N, 1))
x = torch.tensor(x, dtype=torch.float32)

In [ ]:
class LikelihoodNormalDist(nn.Module):
  def __init__(self, mu, sigma):
    super(LikelihoodNormalDist, self).__init__()
    self.mu = nn.Parameter(torch.tensor(mu, dtype=torch.float32))
    self.sigma = nn.Parameter(torch.tensor(sigma, dtype=torch.float32))
    
  def forward(self, x):
    return torch.sqrt(1 / (2 * np.pi * self.sigma**2)) * torch.exp(- (x - self.mu)**2 / (2 * self.sigma**2))

In [ ]:
model = LikelihoodNormalDist(2, 1)
optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
max_epoch = 10000
m, s = np.zeros(max_epoch), np.zeros(max_epoch)
for epoch in range(max_epoch):
  likelihood = model(x)
  log_likelihood = torch.log(likelihood)
  loss = - torch.mean(log_likelihood)
  # print(f'epoch: {epoch} / loss: {loss.item():.4f}')
  
  m[epoch] = model.mu.data.item()
  s[epoch] = model.sigma.data.item()
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
print(f'estimated mean: {model.mu.data.item():.4f}')
print(f'estimated std: {model.sigma.data.item():.4f}')

In [ ]:
def mesh_plot(X, Y, xlim=2, ylim=2):
  fig = plt.figure()
  ax = fig.add_subplot(111)
  plt.plot(X, Y, ls='None', marker='.')
  plt.xlim(-xlim, xlim)
  plt.ylim(-ylim, ylim)
  plt.axvline(x=0, color='black')
  plt.axhline(y=0, color='black')
  ax.set_aspect('equal', adjustable='box')
  plt.show()

In [ ]:
mesh_plot(m, s, xlim=5, ylim=5)